![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 67.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a8ffbf12ae22a3f6441144b2897e9bdebbc081503c96b2af1018cf052deca50a
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


### Create a spark session and import the required libraries

In [75]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

In [76]:
spark = SparkSession.builder.appName('FinalSpamClassifier').getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [77]:
df = spark.read.option("header", "false") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv("/content/SMSSpamCollection.txt") \
    .withColumnRenamed("_c0", "class") \
    .withColumnRenamed("_c1", "text")

### Print the schema

In [78]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [10]:
# I do it in read  data part at first (label,sms) -> (class,text)

### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [79]:
df.show(10,truncate=True)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [80]:
df.show(10,truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [81]:
df2 = df.withColumn('length',length(df['text']))

### Show the new dataframe

In [82]:
df2.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [83]:
from pyspark.sql.functions import col, avg

df2.groupby('class').agg(avg("length").alias("average length")).show()

+-----+-----------------+
|class|   average length|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [84]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import VectorAssembler

In [85]:
TextTokenizer = Tokenizer(inputCol="text", outputCol="text_token")

In [86]:
TextStopRemove = StopWordsRemover(inputCol='text_token',outputCol='stop_token')

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [87]:
CountVec = CountVectorizer(inputCol='stop_token',outputCol='count_vec')

In [88]:
text_idf = IDF(inputCol="count_vec", outputCol="tf_idf")

In [89]:
class_to_index = StringIndexer(inputCol='class',outputCol='label')

In [90]:
VecAssem_text = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [91]:
from pyspark.ml.classification import NaiveBayes
NB = NaiveBayes()

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [92]:
from pyspark.ml import Pipeline

In [93]:
pl = Pipeline(stages=[TextTokenizer,TextStopRemove,CountVec,text_idf,class_to_index ,VecAssem_text,NB ])

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [94]:
(trainDF,testDF) = df2.randomSplit([0.7,0.3])

### Fit your Pipeline model to the training data

In [95]:
model=pl.fit(trainDF)

### Perform predictions on tests dataframe

In [96]:
df3=model.transform(testDF)

### Print the schema of the prediction dataframe

In [97]:
df3.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- text_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- count_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [98]:
df3.show(10)

+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|class|                text|length|          text_token|          stop_token|           count_vec|              tf_idf|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|  ham| and  picking the...|    41|[, and, , picking...|[, , picking, var...|(10810,[3,623,267...|(10810,[3,623,267...|  0.0|(10811,[3,623,267...|[-262.62498063583...|[1.0,3.3687372797...|       0.0|
|  ham| came to look at ...|   103|[, came, to, look...|[, came, look, fl...|(10810,[3,11,165,...|(10810,[3,11,165,...|  0.0|(10811,[3,11,165,...|[-747.09489141897...|[1.0,8.7281105425...|       0.0|


## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [99]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [100]:
# From Spark Documentation i get these three lines of code 
eval = MulticlassClassificationEvaluator()
eval.setPredictionCol("prediction")

MulticlassClassificationEvaluator_bdcdcca16440

In [101]:
eval.evaluate(df3)

0.9769052785268896

# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)